In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [44]:
#foursquare account details
CLIENT_ID = 'your-client-id' # your Foursquare ID
CLIENT_SECRET = 'you-client-secret' # your Foursquare Secret
VERSION = '20200816'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-id
CLIENT_SECRET:you-client-secret


In [3]:
#convert conrad hotel NY location to lat long
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [4]:
#define a query to search for chinese food within 1000 meters of Conrad Hotel
search_query = 'Italian'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PHB3YVT4QSNINECCESUTGTC5B32D2NHA0FIN1PYAEIPZGAZI&client_secret=DOW0ZYPHQT0LJJ1CM4SIH3G50PXCXNXQZZEHCU3G4BKAOVF0&ll=40.7149555,-74.0153365&v=20200610&query=Italian&radius=500&limit=30'

In [5]:
results = requests.get(url).json()

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

c:\users\murtaza\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1597572061,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1597572061,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1597572061,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [13]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Exploring a restaurant

In [21]:
venue_id = '3fd66200f964a520f4e41ee3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3?client_id=PHB3YVT4QSNINECCESUTGTC5B32D2NHA0FIN1PYAEIPZGAZI&client_secret=DOW0ZYPHQT0LJJ1CM4SIH3G50PXCXNXQZZEHCU3G4BKAOVF0&v=20200610'

In [22]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '3fd66200f964a520f4e41ee3',
 'name': 'Ecco',
 'contact': {'phone': '2122277074',
  'formattedPhone': '(212) 227-7074',
  'facebook': '496751683729009',
  'facebookUsername': 'EccoRestaurant',
  'facebookName': 'Ecco'},
 'location': {'address': '124 Chambers St',
  'crossStreet': 'btwn Church St & W Broadway',
  'lat': 40.71533713859952,
  'lng': -74.00884766217825,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71533713859952,
    'lng': -74.00884766217825},
   {'label': 'entrance', 'lat': 40.715202, 'lng': -74.008779}],
  'postalCode': '10007',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['124 Chambers St (btwn Church St & W Broadway)',
   'New York, NY 10007',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/ecco/3fd66200f964a520f4e41ee3',
 'categories': [{'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restaurants',
   'shortName': 'Italian',
   '

In [23]:
#ratings for the restaurant:
result['response']['venue']['rating']

7.3

# Exploring a user

In [ ]:
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

In [31]:
#get user tips
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

# Explore a location

In [38]:
#co-ordinates of Ecco Hotel to explore location
latitude = 40.715337
longitude = -74.008848

In [39]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d5f24ec09484500079aee00',
  'name': 'Los Tacos No. 1',
  'location': {'address': '136 Church St',
   'lat': 40.714267,
   'lng': -74.008756,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.714267,
     'lng': -74.008756}],
   'distance': 119,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['136 Church St',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '2180700',
   'url': 'https://www.seamless.com/menu/los-tacos-no-1-tribeca-136-church-st-new-york/21

In [41]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

c:\users\murtaza\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United Sta...",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839...",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007,...",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",at Church St,NaN,4c154c9a77cea593c401d260
3,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051...",202,10007,US,New York,NY,United States,[83 Murray St (btwn Greenwich St & W Broadway)...,btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
4,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374...",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), N...",btwn W Broadway & Church St,NaN,4a8f2f39f964a520471420e3
5,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.71577306392837...",88,10007,US,New York,NY,United States,[148 Chambers St (btwn West Broadway & Hudson ...,btwn West Broadway & Hudson St,NaN,4adcf23cf964a520cc6221e3
6,Whole Foods Market,Grocery Store,270 Greenwich Street,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",214,10007,US,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",at Warren St,Tribeca,49bc3b0af964a52020541fe3
7,Weather Up,Cocktail Bar,159 Duane St,40.716741,-74.008666,"[{'label': 'display', 'lat': 40.71674084163369...",157,10013,US,New York,NY,United States,"[159 Duane St (btwn Hudson St. & W Broadway), ...",btwn Hudson St. & W Broadway,NaN,4cd89eeb6e8b5941660c64d2
8,Nish Nūsh,Falafel Restaurant,88 Reade St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416...",97,10013,US,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 1001...",at Church St,NaN,50ba9119e4b071a4bae6dc10
9,Mulberry & Vine,Café,73 Warren St,40.715177,-74.010227,"[{'label': 'display', 'lat': 40.71517693966315...",117,10007,US,New York,NY,United States,"[73 Warren St (B/t West Broadway & Greenwich),...",B/t West Broadway & Greenwich,NaN,5171b5cc011cef9833bbb787


In [43]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map